In [1]:
import pandas as pd
import json
import random


In [2]:
csv_file="/Users/sgdowd/Documents/1_EODH/data/global_power_plant_database_with_assetcloss.csv"

In [3]:
df = pd.read_csv(csv_file)

/var/folders/5r/p7wfbbqj6ydb6pgtnh3vbz_w0000gq/T/ipykernel_54498/298181477.py:1: DtypeWarning: Columns (6,12,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [4]:
locations=["Europe","North America","South America","Asia","Africa","Oceania","Global"]
# Choose location at random
def choose_location():
    random_no = random.randint(0,6)
    return locations[random_no]

In [5]:
choose_location()

'Oceania'

In [6]:
def get_type(asset_class):
    match asset_class:
        case "PowerGeneratingAsset":
            asset_type = "Coal"
        case "ThermalPowerGeneratingAsset":
            asset_type = "Gas"
        case "RealEstateAsset":
            asset_type = "Buildings/Industrial"
        case "IndustrialActivity":
            asset_type = "Construction"
    return asset_type

In [7]:
def create_return_json(no_of_rows:int,asset_class:str):
    geojson = {
    "type": "FeatureCollection",
    "features": [],
    "properties": {
        "include_asset_level": True,
        "include_calc_details": True,
        "include_measures": True,
        "years": [2030, 2040, 2050],
        "scenarios": ["ssp126", "ssp245","ssp585"]
    }
    }
    for index, row in df[0:no_of_rows].iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "asset_class": asset_class,
                "type": get_type(asset_class),
                "location": 'Asia',#choose_location(),
            },
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]  # Assuming these columns exist
            }
        }
        geojson['features'].append(feature)
    geojson_string = json.dumps(geojson)#.replace('"', '\\"')
    output_dict  ={
    "inputs": {
        "workspace": "ddowding",
        "json_string":geojson_string
    }}
    with open(f'data/output_{asset_class}_{no_of_rows}.json', 'w') as f:
        json.dump(output_dict, f)
    return geojson_string

In [8]:
asset_classes = ["PowerGeneratingAsset","ThermalPowerGeneratingAsset","RealEstateAsset","IndustrialActivity"]
for asset_class in asset_classes:
    for no_of_rows in [10,500]:
        create_return_json(no_of_rows,asset_class)

return_geojson = create_return_json(500,"PowerGeneratingAsset")